In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

#Set the seed for better comparison
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Load CIFAR 10 dataset

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

#Get the CIFAR datasets
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


train, validation = random_split(train_data, [int(0.8 * len(train_data)), len(train_data) - int(0.8 * len(train_data))])


trainloader = DataLoader(train, batch_size=128)
valloader = DataLoader(validation, batch_size=128,)
testloader = DataLoader(test, batch_size=128)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print("Train Size: "+str(len(train)))
print("Validation Size: "+str(len(validation)))
print("Test Size: "+str(len(test)))

# ResNet-18

Define model

In [ ]:
def trainResNet18(learningRate, optimizerName, Residual, patience=3, epochs=15):

  #Initialize the resnet 18 model, setting the pretrained weifghts to false, ensuring the model has not seen the CIFAR-10 dataset before
  model = models.resnet18(pretrained=False)

  #This portion of the code removes the residual links if that has been selected for the gridsearch
  if not Residual:
    for layer in model.children():
      if isinstance(layer, models.resnet.BasicBlock):
        layer.downsample = None
        layer.conv2 = nn.Conv2d(layer.conv2.in_channels, layer.conv2.out_channels, kernel_size=3, stride=1, padding=1, bias=False)

  #Define the model, setting the fully connected layer to 10 to handle the 10 classes in the CIFAR-10 dataset. The fully connected layer is the layer that will ultimately decide what class a input belongs, so it needs to have the same number of nodes as the number of classes
  model.fc = nn.Linear(model.fc.in_features, 10)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  #Define the loss function and set up the optomizer based on which has been passed to the model from the gridsearch. CrossEntropyLoss also known as log loss is being used in these models because it works well in stochastic gradient descent optimizers like the ones i have chosen below. The optimizers also needs to define the learning rates, which have also been passed to it
  lossFunction = nn.CrossEntropyLoss()
  if optimizerName == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=learningRate)
  elif optimizerName == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=learningRate, momentum=0.9)

  #Here i am defining some variable to store the metrics so that i can plot them later
  bestValidationAccuracy = 0
  epochsNoImprove = 0
  trainingLosses, validationAccuracies = [], []
  trainingAccuracies = []

  #The for loop is constrained by the number of epochs. IN each epoch the model trains on the images and labels, calculates the metrics and then validates the model on the validation data set. i make sure that the test set is not seen by the model here so that the training is not infected
  for epoch in range(epochs):
    print("Epoch: ", epoch)
    model.train()
    runningLoss = 0.0
    correctTrain = 0
    totalTrain = 0

    #For each batch of image and label in the trainloader the model is loading it to the gpu for faster training, it then sets the gradients to zero to avoid accumulation. It then predicts the labels for the images, storing the results in the outputs. The loss is then calculated using the loss that was defined earlier ...
    #The loss is calculated using by comparing the outputs/ predictions made by the model on the training set and the true labels. This essentially calculates the errors. The loss gradients is then used in the backpropogation step, which updates all the weights in the optimal direction
    for images, labels in trainloader:
      images, labels = images.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = model(images)
      loss = lossFunction(outputs, labels)
      loss.backward()
      optimizer.step()

      #I will now calculate all of the metrics for the batch
      runningLoss = runningLoss + loss.item()
      _, predicted = torch.max(outputs.data, 1)
      totalTrain = totalTrain + labels.size(0)
      correctTrain = correctTrain + (predicted == labels).sum().item()

    #Calculate all the metrics for this epoch, store them so that i can plot them later and analyse what happened during the training process
    epochLoss = runningLoss / len(trainloader)
    trainingLosses.append(epochLoss)
    trainAccuracy = correctTrain / totalTrain
    trainingAccuracies.append(trainAccuracy)

    #Using a seperate validation set so that i dont infect the training data i need to validate the models performance. The model technically has not been trained on the validation set so this is unseen data, however because the model will be adjusted based on the results of this validation set the model will fit to this dataset thus infecting it.
    model.eval()
    correctVal = 0
    totalVal = 0
    with torch.no_grad():
      for images, labels in valloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        totalVal = totalVal + labels.size(0)
        correctVal = correctVal + (predicted == labels).sum().item()

    validationAccuracy = correctVal / totalVal
    validationAccuracies.append(validationAccuracy)

    #Apply early stopping by checking if the validation acccuracy is not improving for the length of the patience. Also if the model is has the best validation accuracy save the weights to use later to avoid having to retrain. This is to improve computational efficiency as there is no point in continuing the training of a model that is not performing well
    if validationAccuracy > bestValidationAccuracy:
      bestValidationAccuracy = validationAccuracy
      torch.save(model.state_dict(), 'best_ResNet18.pth')
      epochsNoImprove = 0
    else:
      epochsNoImprove = epochsNoImprove + 1
      if epochsNoImprove >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

  return trainingLosses, trainingAccuracies, validationAccuracies, bestValidationAccuracy

#Define the paramters to use in the grid search, i have not used very many due to resource limitations as it takes around 30 minutes to train one iteration of the params
learningRates = [0.01, 0.001]
optimizers = ['Adam', 'SGD']
residuals = [True, False]

bestAccuracy = 0
bestParams = {}
allTrainingLosses = {}
allValidationAccuracies = {}

#Perform the gridsearch by iterating through all of the options using itertools to ensure every combination of hyperparameters is covered. Store the params of the model with the best validation accuracy, as this will likely be the best perfrorming model.
for lr, opt, residual in itertools.product(learningRates, optimizers, residuals):
  trainLosses, trainAccuracies, valAccuracies, valAccuracy = trainResNet18(lr, opt, residual, patience=3)

  if valAccuracy > bestAccuracy:
    bestAccuracy = valAccuracy
    bestParams = {'learning_rate': lr, 'optimizer': opt, 'use_residual': residual}

  paramStr = f"lr={lr}, opt={opt}, residual={residual}"
  allTrainingLosses[paramStr] = trainLosses
  allValidationAccuracies[paramStr] = valAccuracies

In [ ]:
print("Best Validation Accuracy:", bestAccuracy)
print("Best Hyperparameters:", bestParams)

# Plot all grid search training loss curves
plt.figure(figsize=(12, 6))
for key, losses in allTrainingLosses.items():
  plt.plot(losses, label=key)
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Training Loss ResNet-18")
plt.legend()
plt.show()

# Plot all grid search validation accuracy curves
plt.figure(figsize=(12, 6))
for key, accuracies in allValidationAccuracies.items():
  plt.plot(accuracies, label=key)
plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")
plt.title("Validation Accuracy for ResNet-18")
plt.legend()
plt.show()

In [ ]:
def evaluateResNet18(testloader):
  # Load the best model weights
  model = models.resnet18(pretrained=False)
  model.fc = nn.Linear(model.fc.in_features, 10)
  model.load_state_dict(torch.load('best_ResNet18.pth'))
  model.eval()

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  predictions = []
  pLabels = []

  with torch.no_grad():
    for images, labels in testloader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)

      predictions.extend(predicted.cpu().numpy())
      pLabels.extend(labels.cpu().numpy())

  # Calculate overall accuracy
  testAccuracy = accuracy_score(pLabels, predictions)
  print(f"Test Accuracy: {testAccuracy:.4f}")

  # Confusion Matrix
  cm = confusion_matrix(pLabels, predictions)
  plt.figure(figsize=(8, 6))
  sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
  plt.xlabel("Predicted Label")
  plt.ylabel("True Label")
  plt.title("Confusion Matrix for ResNet-18")
  plt.show()

  # Classification Report
  print("Classification Report for ResNet18:")
  print(classification_report(pLabels, predictions))

# Assuming `testloader` is the DataLoader for your test dataset
evaluateResNet18(testloader)

# AlexNet

In [ ]:
def trainAlexNet(learningRate, optimizerName, weight_decay, patience=3, epochs=15):
  model = models.alexnet(pretrained=False)
  model.features[0] = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)

  model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  lossFunction = nn.CrossEntropyLoss()
  if optimizerName == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=learningRate)
  elif optimizerName == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=learningRate, momentum=0.9)

  bestValidationAccuracy = 0
  epochsNoImprove = 0
  trainingLosses, validationAccuracies = [], []
  trainingAccuracies = []

  for epoch in range(epochs):
    print("Epoch: " + str(epoch))
    model.train()
    runningLoss = 0.0
    correctTrain = 0
    totalTrain = 0

    for images, labels in trainloader:
      images, labels = images.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = model(images)
      loss = lossFunction(outputs, labels)
      loss.backward()
      optimizer.step()

      runningLoss = runningLoss + loss.item()
      _, predicted = torch.max(outputs.data, 1)
      totalTrain = totalTrain + labels.size(0)
      correctTrain = correctTrain + (predicted == labels).sum().item()

    epochLoss = runningLoss / len(trainloader)
    trainingLosses.append(epochLoss)
    trainAccuracy = correctTrain / totalTrain
    trainingAccuracies.append(trainAccuracy)

    model.eval()
    correctVal = 0
    totalVal = 0
    with torch.no_grad():
      for images, labels in valloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        totalVal = totalVal + labels.size(0)
        correctVal = correctVal + (predicted == labels).sum().item()

    validationAccuracy = correctVal / totalVal
    validationAccuracies.append(validationAccuracy)

    if validationAccuracy > bestValidationAccuracy:
      bestValidationAccuracy = validationAccuracy
      torch.save(model.state_dict(), 'best_AlexNet.pth')
      epochsNoImprove = 0
    else:
      epochsNoImprove = epochsNoImprove + 1
      if epochsNoImprove >= patience:
        print("Early stopping at epoch " + str(epoch+1))
        break

  return trainingLosses, trainingAccuracies, validationAccuracies, bestValidationAccuracy

learningRates = [0.01, 0.001]
optimizers = ['Adam', 'SGD']
weightDecay = [1e-2,1e-4]

bestAccuracy = 0
bestParams = {}
allTrainingLosses = {}
allValidationAccuracies = {}

for lr, opt, wd in itertools.product(learningRates, optimizers, weightDecay):
  print(f"Training with lr={lr}, optimizer={opt}, weight_decay={wd}")
  trainLosses, trainAccuracies, valAccuracies, valAccuracy = trainAlexNet(lr, opt, wd, patience=3)

  if valAccuracy > bestAccuracy:
    bestAccuracy = valAccuracy
    bestParams = {'learning_rate': lr, 'optimizer': opt, 'weight_decay': wd}

  paramStr = f"lr={lr}, opt={opt}, weight_decay={wd}"
  allTrainingLosses[paramStr] = trainLosses
  allValidationAccuracies[paramStr] = valAccuracies

In [ ]:
print("Best Validation Accuracy:", bestAccuracy)
print("Best Hyperparameters:", bestParams)

plt.figure(figsize=(12, 6))
for key, losses in allTrainingLosses.items():
    plt.plot(losses, label=key)
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Training Loss for Different Hyperparameter Combinations")
plt.legend()
plt.show()
s
plt.figure(figsize=(12, 6))
for key, accuracies in allValidationAccuracies.items():
    plt.plot(accuracies, label=key)
plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")
plt.title("Validation Accuracy for Different Hyperparameter Combinations")
plt.legend()
plt.show()

In [ ]:
def evaluateAlexNet(testloader):
  model = models.alexnet(pretrained=False)
  model.features[0] = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)

  #There are 10 classes in CIFAR-10 so ensure this is reflected in the model
  model.classifier[6] = nn.Linear(model.classifier[6].in_feapredictions)
  model.load_state_dict(torch.load('best_AlexNet.pth'))
  model.eval()

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  predictions = []
  all_labels = []

  with torch.no_grad():
    for images, labels in testloader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)

      predictions.extend(predicted.cpu().numpy())
      all_labels.extend(labels.cpu().numpy())

  test_accuracy = accuracy_score(all_labels, predictions)
  print(f"Test Accuracy: {test_accuracy:.4f}")

  cm = confusion_matrix(all_labels, predictions)
  plt.figure(figsize=(8, 6))
  sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
  plt.xlabel("Predicted Label")
  plt.ylabel("True Label")
  plt.title("Confusion Matrix for AlexNet")
  plt.show()

  print("Classification Report for AlexNet:")
  print(classification_report(all_labels, predictions))

evaluateAlexNet(testloader)

# MobileNet

In [ ]:
def trainMobileNet(learningRate, optimizerName, dropout, patience=3, epochs=15):
  model = models.mobilenet_v2(pretrained=False)

  model.classifier = nn.Sequential(
      nn.Dropout(p=dropout),
      nn.Linear(model.classifier[1].in_features, 10)
  )

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  lossFunction = nn.CrossEntropyLoss()
  if optimizerName == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=learningRate)
  elif optimizerName == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=learningRate, momentum=0.9)

  bestValidationAccuracy = 0
  epochsNoImprove = 0
  trainingLosses, validationAccuracies = [], []
  trainingAccuracies = []

  for epoch in range(epochs):
    print("Epoch: " + str(epoch))
    model.train()
    runningLoss = 0.0
    correctTrain = 0
    totalTrain = 0

    for images, labels in trainloader:
      images, labels = images.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = model(images)
      loss = lossFunction(outputs, labels)
      loss.backward()
      optimizer.step()

      runningLoss = runningLoss + loss.item()
      _, predicted = torch.max(outputs.data, 1)
      totalTrain = totalTrain + labels.size(0)
      correctTrain = correctTrain + (predicted == labels).sum().item()

    epochLoss = runningLoss / len(trainloader)
    trainingLosses.append(epochLoss)
    trainAccuracy = correctTrain / totalTrain
    trainingAccuracies.append(trainAccuracy)

    model.eval()
    correctVal = 0
    totalVal = 0
    with torch.no_grad():
      for images, labels in valloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        totalVal = totalVal + labels.size(0)
        correctVal = correctVal + (predicted == labels).sum().item()

    validationAccuracy = correctVal / totalVal
    validationAccuracies.append(validationAccuracy)

    if validationAccuracy > bestValidationAccuracy:
      bestValidationAccuracy = validationAccuracy
      torch.save(model.state_dict(), 'best_ResNet18.pth')
      epochsNoImprove = 0
    else:
      epochsNoImprove = epochsNoImprove + 1
      if epochsNoImprove >= patience:
        print("Early stopping at epoch " + str(epoch+1))
        break

  return trainingLosses, trainingAccuracies, validationAccuracies, bestValidationAccuracy


learningRates = [0.01,0.001]
optimizers = ['Adam', 'SGD']
dropouts = [0.25,0.3]


bestAccuracy = 0
bestParams = {}
allTrainingLosses = {}
allValidationAccuracies = {}

for lr, opt, dropout in itertools.product(learningRates, optimizers, dropouts):
  print(f"Training with lr={lr}, optimizer={opt}, dropout={dropout}")
  trainLosses, trainAccuracies, valAccuracies, valAccuracy = trainMobileNet(lr, opt, dropout, patience=3)

  if valAccuracy > bestAccuracy:
    bestAccuracy = valAccuracy
    bestParams = {'learning_rate': lr, 'optimizer': opt, 'dropout': dropout}

  paramStr = f"lr={lr}, opt={opt}, dropout={dropout}"
  allTrainingLosses[paramStr] = trainLosses
  allValidationAccuracies[paramStr] = valAccuracies

In [ ]:
print("Best Validation Accuracy:", bestAccuracy)
print("Best Hyperparameters:", bestParams)

plt.figure(figsize=(12, 6))
for key, losses in allTrainingLosses.items():
  plt.plot(losses, label=key)
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Training Loss for MobileNet")
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
for key, accuracies in allValidationAccuracies.items():
  plt.plot(accuracies, label=key)
plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")
plt.title("Validation Accuracy for MobileNet")
plt.legend()
plt.show()

In [ ]:
def evaluateMobileNet(testloader):
  #Use the best weights
  model = models.mobilenet_v2(pretrained=False)
  model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
  model.load_state_dict(torch.load('best_MobileNet.pth'))
  model.eval()

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  predictions = []
  pLabels = []

  with torch.no_grad():
    for images, labels in testloader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)

      predictions.extend(predicted.cpu().numpy())
      pLabels.extend(labels.cpu().numpy())

  test_accuracy = accuracy_score(pLabels, predictions)
  print(f"Test Accuracy: {test_accuracy:.4f}")

  cm = confusion_matrix(pLabels, predictions)
  plt.figure(figsize=(8, 6))
  sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
  plt.xlabel("Predicted Label")
  plt.ylabel("True Label")
  plt.title("Confusion Matrix for MobileNet")
  plt.show()

  print("Classification Report for MobileNet:")
  print(classification_report(pLabels, predictions))

evaluateMobileNet(testloader)

# Data Augmentation experiment with ResNet-18

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train, validation = random_split(train_data, [int(0.8 * len(train_data)), len(train_data) - int(0.8 * len(train_data))])

trainloader = DataLoader(train, batch_size=128, shuffle=True)
valloader = DataLoader(validation, batch_size=128, shuffle=False)
testloader = DataLoader(test, batch_size=128, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print("Train Size: "+str(len(train)))
print("Validation Size: "+str(len(validation)))
print("Test Size: "+str(len(test)))

In [ ]:
def trainResNet18(learningRate, optimizerName, Residual, patience=3, epochs=15):
  model = models.resnet18(pretrained=False)

  if not Residual:
    for layer in model.children():
      if isinstance(layer, models.resnet.BasicBlock):
        layer.downsample = None
        layer.conv2 = nn.Conv2d(layer.conv2.in_channels, layer.conv2.out_channels, kernel_size=3, stride=1, padding=1, bias=False)

  model.fc = nn.Linear(model.fc.in_features, 10)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  lossFunction = nn.CrossEntropyLoss()
  if optimizerName == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=learningRate)
  elif optimizerName == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=learningRate, momentum=0.9)

  bestValidationAccuracy = 0
  epochsNoImprove = 0
  trainingLosses, validationAccuracies = [], []
  trainingAccuracies = []

  for epoch in range(epochs):
    print("Epoch: " + str(epoch))
    model.train()
    runningLoss = 0.0
    correctTrain = 0
    totalTrain = 0

    for images, labels in trainloader:
      images, labels = images.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = model(images)
      loss = lossFunction(outputs, labels)
      loss.backward()
      optimizer.step()

      runningLoss = runningLoss + loss.item()
      _, predicted = torch.max(outputs.data, 1)
      totalTrain = totalTrain + labels.size(0)
      correctTrain = correctTrain + (predicted == labels).sum().item()

    epochLoss = runningLoss / len(trainloader)
    trainingLosses.append(epochLoss)
    trainAccuracy = correctTrain / totalTrain
    trainingAccuracies.append(trainAccuracy)

    model.eval()
    correctVal = 0
    totalVal = 0
    with torch.no_grad():
      for images, labels in valloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        totalVal = totalVal + labels.size(0)
        correctVal = correctVal + (predicted == labels).sum().item()

    validationAccuracy = correctVal / totalVal
    validationAccuracies.append(validationAccuracy)

    if validationAccuracy > bestValidationAccuracy:
      bestValidationAccuracy = validationAccuracy
      torch.save(model.state_dict(), 'best_ResNet18.pth')
      epochsNoImprove = 0
    else:
      epochsNoImprove = epochsNoImprove + 1
      if epochsNoImprove >= patience:
        print("Early stopping at epoch " + str(epoch+1))
        break

  return trainingLosses, trainingAccuracies, validationAccuracies, bestValidationAccuracy

learningRates = [0.01, 0.001]
optimizers = ['Adam', 'SGD']
residuals = [True, False]

bestAccuracy = 0
bestParams = {}
allTrainingLosses = {}
allValidationAccuracies = {}

for lr, opt, residual in itertools.product(learningRates, optimizers, residuals):
  trainLosses, trainAccuracies, valAccuracies, valAccuracy = trainResNet18(lr, opt, residual, patience=3)

  if valAccuracy > bestAccuracy:
    bestAccuracy = valAccuracy
    bestParams = {'learning_rate': lr, 'optimizer': opt, 'use_residual': residual}

  paramStr = f"lr={lr}, opt={opt}, residual={residual}"
  allTrainingLosses[paramStr] = trainLosses
  allValidationAccuracies[paramStr] = valAccuracies

In [ ]:
print("Best Validation Accuracy:", bestAccuracy)
print("Best Hyperparameters:", bestParams)

plt.figure(figsize=(12, 6))
for key, losses in allTrainingLosses.items():
  plt.plot(losses, label=key)
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Training Loss ResNet-18 with Augmentation")
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
for key, accuracies in allValidationAccuracies.items():
  plt.plot(accuracies, label=key)
plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")
plt.title("Validation Accuracy for ResNet-18 with Augmentation")
plt.legend()
plt.show()

In [ ]:
def evaluateResNet18(testloader):
  model = models.resnet18(pretrained=False)
  model.fc = nn.Linear(model.fc.in_features, 10)
  model.load_state_dict(torch.load('best_ResNet18.pth'))
  model.eval()

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  predictions = []
  pLabels = []

  with torch.no_grad():
    for images, labels in testloader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)

      predictions.extend(predicted.cpu().numpy())
      pLabels.extend(labels.cpu().numpy())

  testAccuracy = accuracy_score(pLabels, predictions)
  print(f"Test Accuracy: {testAccuracy:.4f}")

  cm = confusion_matrix(pLabels, predictions)
  plt.figure(figsize=(8, 6))
  sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
  plt.xlabel("Predicted Label")
  plt.ylabel("True Label")
  plt.title("Confusion Matrix for ResNet-18 with Augmentation")
  plt.show()

  print("Classification Report for ResNet18 with Augmentation:")
  print(classification_report(pLabels, predictions))

evaluateResNet18(testloader)